In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.2/934.2 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 4.2 MB/s eta 0:00

In [ ]:
from darts.models.forecasting.dlinear import DLinearModel
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

In [ ]:
processed_data_dir = '/content/drive/MyDrive/M5-A1/2. data/processed/'
processed_data_dir_base = '/content/drive/MyDrive/M5-thesis/M5-FITS/processed/'
ver, KKK = 'priv', 0
START_TRAIN = 41
END_TRAIN   = 1941 - 28*KKK
P_HORIZON   = 28
USE_AUX     = False
TARGET = 'sales'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'

remove_features = ['id','cat_id', 'state_id','store_id','dept_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std']

In [ ]:
global_df = pd.read_pickle(processed_data_dir_base + 'raw_data.pkl')

In [ ]:
def reduce_mem_usage(df, verbose=True):
    """
    from M5 Forecast: Keras with Categorical Embeddings V2
    https://www.kaggle.com/mayer79/m5-forecast-keras-with-categorical-embeddings-v2
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'\
                      .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df = reduce_mem_usage(df)
df

Mem. usage decreased to 2230.93 Mb (10.9% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,...,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,0,0.459961,...,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,0,1.559570,...,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,3.169922,...,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,5.980469,...,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,0,0.700195,...,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,2.980469,...,0,0,0,19,24,6,5,3,6,1
47735393,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,2.480469,...,0,0,0,19,24,6,5,3,6,1
47735394,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,3.980469,...,0,0,0,19,24,6,5,3,6,1
47735395,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1969,NaN,230,1.280273,...,0,0,0,19,24,6,5,3,6,1


## Data preparation

In [ ]:
input_len=1913
train_start = 0
validation_start = 1913
test_start = 1941
horizon = 28 #days to predict

In [ ]:
from darts.timeseries import TimeSeries

exog_variables_simple = ['snap_count', 'is_event', 'tm_w_end', 'tm_d', 'tm_w', 'tm_dw']

def prepare_dlinear_data(data, override_features=[], simple=False, to_ts=True):
  data['snap_CA'] = data['snap_CA'].astype(int)
  data['snap_TX'] = data['snap_TX'].astype(int)
  data['snap_WI'] = data['snap_WI'].astype(int)
  req_cols = ['d', 'sales']
  features = exog_variables_simple if len(override_features) == 0 else override_features
  # calendar = data[['d', 'snap_CA', 'snap_TX', 'snap_WI', 'event_name_1', 'event_name_2', 'tm_w_end', 'tm_d', 'tm_w', 'tm_y', 'tm_m', 'tm_wm',  'price_momentum', 'tm_m', 'tm_y', 'sales', 'tm_dw']].copy()
  if simple:
    data['snap_count'] = data[['snap_CA', 'snap_WI', 'snap_TX']].apply(np.sum, axis=1)
    data['is_event_1'] = [isinstance(x , str)*1 for x in data['event_name_1']]
    data['is_event_2'] = [isinstance(x , str)*1 for x in data['event_name_2']]
    data['is_event'] = data[['is_event_1', 'is_event_2']].apply(np.sum, axis=1)
    data['is_event'] = np.where(data['is_event']>0,1,0)
  else:
    data['event_name_1'] = data['event_name_1'].astype('category')
    data['event_name_2'] = data['event_name_2'].astype('category')
    data['event_type_1'] = data['event_type_1'].astype('category')
    data['event_type_2'] = data['event_type_2'].astype('category')

  data = data[req_cols + features]
  exog_fit = data[data['d'] <= test_start]
  exog_fit.index = range(len(exog_fit))
  # exog_fit.index = exog_fit['d']
  exog_fit.drop(['d', 'sales'], axis=1, inplace=True)

  exog_pred = data.iloc[train_start:,:].copy()
  exog_pred.index = range(len(exog_pred))
  # exog_pred.index = exog_pred['d']
  exog_pred.drop(['d', 'sales'], axis=1, inplace=True)

  y_train = data[data['d'] <= test_start]['sales']
  y_train.index = exog_fit.index
  y_train = TimeSeries.from_series(y_train)

  if to_ts:
    exog_fit = TimeSeries.from_dataframe(exog_fit, value_cols=features)
    exog_pred = TimeSeries.from_dataframe(exog_pred, value_cols=features)

  return y_train, exog_fit, exog_pred

In [ ]:
exclude = ['id', 'price_nunique', 'item_nunique', 'sel_price', 'price_std', 'release', 'item_id', 'd', 'sales', 'event_name_1', 'is_event_1', 'is_event_2', 'event_name_2', 'event_type_1', 'event_type_2', 'price_min', 'price_max', 'snap_CA', 'snap_TX', 'snap_WI']
exclude_like = ['rolling', 'lag', 'mean', 'momentum']
features = [x for x in item_1.columns if x not in exclude]
features = [item for item in features if not any(sub in item for sub in exclude_like)]
y_train, exog_fit, exog_pred = prepare_dlinear_data(item_1, override_features=features, simple=True, to_ts=True)

In [ ]:
features

['sell_price',
 'price_norm',
 'tm_d',
 'tm_w',
 'tm_m',
 'tm_y',
 'tm_wm',
 'tm_dw',
 'tm_w_end',
 'enc_item_id_store_id_std',
 'snap_count',
 'is_event',
 'snap_count',
 'is_event']

## DLinear model playground

In [ ]:
model = DLinearModel(
      input_chunk_length=1800,
      output_chunk_length=28,
      n_epochs=300,
      model_name='dlinear_model',
      random_state=42,
      batch_size=64
  )
model.fit(y_train, past_covariates=exog_fit, future_covariates=exog_pred, verbose=True)
preds = model.predict(28, past_covariates=exog_fit, future_covariates=exog_pred)
preds.values()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | decomposition   | _SeriesDecomp    | 0      | train
6 | linear_seasonal | Linear           | 1.3 M  | train
7 | linear_trend    | Linear           | 1.3 M  | train
8 | linear_fut_cov  | Linear           | 13     | train
--

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

array([[10.88120876],
       [ 3.88806888],
       [ 4.02978833],
       [ 9.56027578],
       [ 4.51059006],
       [ 7.55284307],
       [ 9.19423064],
       [ 9.62075796],
       [ 7.22178853],
       [11.24170078],
       [11.47195065],
       [ 9.46905466],
       [ 7.31809063],
       [ 4.81370251],
       [ 5.04167176],
       [ 4.43301   ],
       [ 8.83879812],
       [12.1231912 ],
       [ 5.82225344],
       [13.13977988],
       [18.26988783],
       [ 6.40536858],
       [ 1.27801572],
       [ 4.58550847],
       [ 4.59268275],
       [ 6.14071865],
       [17.97838266],
       [11.6303451 ]])

In [ ]:
del df, model
gc.collect()

170

In [ ]:
eval = pd.read_csv('/content/drive/MyDrive/data/m5-data/sales_test_evaluation.csv')

In [ ]:
temp = eval[eval['item_id'] == 'HOBBIES_1_008']
eval_item = temp[temp['store_id'] == 'CA_1']
eval_item.filter(like='d_').values[0]

array([ 1,  3,  0, 22, 11, 11, 12,  7,  9,  3,  2, 12, 13, 28, 18,  0,  3,
        5, 14, 22, 15, 30,  9, 19,  8,  4, 14, 23])

## Preprocessed data analysis


*   Models can be trained on data aggregated on multiple levels (state, store, category, etc.)
*   Additional features can be added to the model: lags and rolling means


The purpose of this section is to analyse the M5 competition winning modelling - features and modelling strategies



In [ ]:
def get_data_by_store(store, dept):
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)

    df = df[df['d']>=START_TRAIN]

    df = df[(df['store_id']==store) & (df['dept_id']==dept)]

    # df['snap_count'] = df[['snap_CA', 'snap_WI', 'snap_TX']].apply(np.sum, axis=1)
    # df['is_event_1'] = [isinstance(x , str)*1 for x in df['event_name_1']]
    # df['is_event_2'] = [isinstance(x , str)*1 for x in df['event_name_2']]
    # df['is_event'] = df[['is_event_1', 'is_event_2']].apply(np.sum, axis=1)
    # df['is_event'] = np.where(df['is_event']>0,1,0)

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]

    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]

    df = pd.concat([df, df2], axis=1)
    del df2

    df = pd.concat([df, df3], axis=1)
    del df3

    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]

    df = df.reset_index(drop=True)

    return df, features

In [ ]:
input_dir = '/content/drive/MyDrive/data/m5-data/'
dir_ = '/content/drive/MyDrive/M5-A1/' # input only here
processed_data_dir = dir_ + '2. data/processed/'

In [ ]:
import pandas as pd

In [ ]:
dept_id = 'HOBBIES_1'
state = 'CA_1'

In [ ]:
data, features = get_data_by_store(state, dept_id)

In [ ]:
data = data.drop(columns=data.filter(like='tmp').columns)
item_1 = data[data['id'] == 'HOBBIES_1_008_CA_1_evaluation']
item_1.columns

Index(['id', 'd', 'sales', 'item_id', 'release', 'sell_price', 'price_max',
       'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique',
       'item_nunique', 'price_momentum', 'price_momentum_m',
       'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m',
       'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_item_id_store_id_mean',
       'enc_item_id_store_id_std', 'sales_lag_28', 'sales_lag_29',
       'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33',
       'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37',
       'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41',
       'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14',
       'rolling_std_14', 'rolling_mean_30', 'rolling_std_30',
       'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180',
       'rolling_std_180'],
      dtype='object')

In [ ]:
exclude = ['id', 'release', 'item_id', 'd', 'sales', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'price_min', 'price_max', 'snap_CA', 'snap_TX', 'snap_WI']
exclude_like = ['rolling', 'lag_3' 'mean', 'momentum']
features = [x for x in item_1.columns if x not in exclude]
features = [item for item in features if not any(sub in item for sub in exclude_like)]
y_train, exog_fit, exog_pred = prepare_dlinear_data(item_1, override_features=features, simple=True, to_ts=False)

ERROR:darts.timeseries:ValueError: Could not convert integer index to a pd.RangeIndex. Found non-unique step sizes/frequencies: {194, 195, 197, 200, 201, 203, 204, 205, 206, 207, 209, 211, 212, 219, 222, 223, 225, 227, 228, 230, 231, 232, 233, 234, 235, 241, 255, 257, 261, 264, 265, 266, 267, 268, 269, 273, 275, 278, 284, 285, 286, 290, 300, 301, 307, 308, 310, 314, 316, 317, 319, 320, 322, 323, 327, 330, 334, 338, 339, 340, 342, 343, 344, 345, 349, 350, 354, 356, 357, 358, 361, 364, 365, 366, 368, 371, 372, 374, 375, 376, 377, 378, 380, 382, 383, 384, 387, 388, 390, 395, 398, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 415, 416}. If any of those is the actual frequency, try passing it with fill_missing_dates=True and freq=your_frequency.


ValueError: Could not convert integer index to a pd.RangeIndex. Found non-unique step sizes/frequencies: {194, 195, 197, 200, 201, 203, 204, 205, 206, 207, 209, 211, 212, 219, 222, 223, 225, 227, 228, 230, 231, 232, 233, 234, 235, 241, 255, 257, 261, 264, 265, 266, 267, 268, 269, 273, 275, 278, 284, 285, 286, 290, 300, 301, 307, 308, 310, 314, 316, 317, 319, 320, 322, 323, 327, 330, 334, 338, 339, 340, 342, 343, 344, 345, 349, 350, 354, 356, 357, 358, 361, 364, 365, 366, 368, 371, 372, 374, 375, 376, 377, 378, 380, 382, 383, 384, 387, 388, 390, 395, 398, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 415, 416}. If any of those is the actual frequency, try passing it with fill_missing_dates=True and freq=your_frequency.

In [ ]:
model = DLinearModel(
    input_chunk_length=1460,
    output_chunk_length=28,
    n_epochs=50,
    model_name='dlinear_model',
    random_state=42
)
model.fit(y_train, past_covariates=exog_fit, future_covariates=exog_pred, verbose=True)
preds = model.predict(28, past_covariates=exog_fit, future_covariates=exog_pred)
preds.values()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | decomposition   | _SeriesDecomp    | 0      | train
6 | linear_seasonal | Linear           | 3.3 M  | train
7 | linear_trend    | Linear           | 3.3 M  | train
8 | linear_fut_cov  | Linear           | 41     | train
--

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.
ERROR:darts.utils.data.inference_dataset:ValueError: For the given forecasting horizon `n=28`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n <= output_chunk_length` the future covariates must end at time step `1997`, whereas now they end at time step `1969`.


ValueError: For the given forecasting horizon `n=28`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n <= output_chunk_length` the future covariates must end at time step `1997`, whereas now they end at time step `1969`.

## Submission #1
DLinear model with simplified parameters

In [ ]:
global_df = pd.read_pickle(processed_data_dir_base + 'raw_data.pkl')

In [ ]:
df = reduce_mem_usage(df)

Mem. usage decreased to 2230.93 Mb (10.9% reduction)


In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/data/m5-data/sample_submission.csv')

In [ ]:
items = df['id'].unique()
for i in range(len(items)):
  item_id = items[i]
  print(f"Training {item_id}, {i + 1}/{len(items)}")
  item = df[df['id'] == item_id].copy()
  y_train, exog_fit, exog_pred = prepare_dlinear_data(item, simple=True)
  model = DLinearModel(
      input_chunk_length=1913-train_start,
      output_chunk_length=28,
      n_epochs=200,
      model_name='dlinear_model',
      random_state=42,
      batch_size=64
  )
  model.fit(y_train, past_covariates=exog_fit, future_covariates=exog_pred, verbose=False)
  preds = model.predict(28, past_covariates=exog_fit, future_covariates=exog_pred, verbose=False)
  preds = preds.values()

  for i in range(len(preds)):
    col = 'F' + str(i + 1)
    submission.loc[submission['id'] == item_id, col] = preds[i]

  submission[submission['id'] == item_id]
  del model, preds
  del item
  del y_train
  del exog_fit, exog_pred
  gc.collect()

Training HOBBIES_1_008_CA_1_evaluation, 1/30490


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training HOBBIES_1_009_CA_1_evaluation, 2/30490


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training HOBBIES_1_010_CA_1_evaluation, 3/30490


...

Training HOBBIES_1_036_CA_1_evaluation, 12/30490


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training HOBBIES_1_038_CA_1_evaluation, 13/30490


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
submission.to_csv('/content/drive/MyDrive/data/m5-data/submissions/dlinear-simple.csv')


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30497,HOBBIES_1_008_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Submission #2
Simple model with aggregation on store-item type level
Aggregate by:

*   store_id
*   dept_id

#### Calculate weights for each item for disaggregation

Weights should be calculated for the last 28 days of sales.
W = item_sales / agg_category_sales

#### Data structure

`df_train` should have 1941 columns (or N) sales data for each day. `df_calendar` should have 1941 (or N) points of exogenous features (event name, snap_*, tm_w_end, etc.) for each day.




In [ ]:
from darts.models.forecasting.dlinear import DLinearModel
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

In [ ]:
processed_data_dir = '/content/drive/MyDrive/M5-A1/2. data/processed/'
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
ver, KKK = 'priv', 0
START_TRAIN = 41
END_TRAIN   = 1941 - 28*KKK
P_HORIZON   = 28
USE_AUX     = False
TARGET = 'sales'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'

remove_features = ['id','cat_id', 'state_id','store_id','dept_id',
                   'date','wm_yr_wk','d',TARGET]df_
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std']

In [ ]:
global_df = pd.read_pickle(processed_data_dir_base + 'raw_data.pkl')

In [ ]:
from darts.timeseries import TimeSeries

TRAIN_START = 0
TEST_START = 1942
HORIZON = 28

In [ ]:
"""
get df_train and features
"""
def get_data_by_store_id(store_id, dept_id):
  df = global_df.copy()

  store_data = df[(df['store_id'] == store_id) & (df['dept_id'] == dept_id) & (df['d'] >= TRAIN_START)]
  df_train = store_data.pivot(index='id', columns='d', values='sales')
  df_train.fillna(0, inplace=True)
  df_calendar = store_data.groupby(['d']).agg({
    'event_name_1': 'first',
    'event_name_2': 'first',
    'snap_CA': 'first',
    'snap_TX': 'first',
    'snap_WI': 'first',
    'tm_w_end': 'first',
    'tm_dw': 'first',
  }).reset_index()
  df_calendar.index = df_calendar['d']

  del df, store_data
  gc.collect()

  return df_train, df_calendar

In [ ]:
df_train, df_calendar = get_data_by_store_id('CA_1', 'HOBBIES_1')

In [ ]:
df_train.iloc[:, 0:(TEST_START - TRAIN_START - 1)]

d,1,2,3,4,5,6,7,8,9,10,...,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_008_CA_1_evaluation,12.0,15.0,0.0,0.0,0.0,4.0,6.0,5.0,7.0,0.0,...,7.0,0.0,6.0,0.0,15.0,5.0,4.0,1.0,40.0,32.0
HOBBIES_1_009_CA_1_evaluation,2.0,0.0,7.0,3.0,0.0,2.0,3.0,9.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
HOBBIES_1_010_CA_1_evaluation,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0
HOBBIES_1_012_CA_1_evaluation,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
HOBBIES_1_015_CA_1_evaluation,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,15.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOBBIES_1_125_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
HOBBIES_1_232_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,19.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_269_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
class M5DLinear(object):
  def __init__(self, horizon, df_train, df_calendar, epochs):
    self.horizon = horizon
    self.df_train = df_train
    self.df_calendar = df_calendar
    self.exog_fit, self.exog_pred = self.get_exog_variables()
    self.top_level = self.get_top_level()
    self.train_len = len(self.df_train.columns)
    self.epochs = epochs

  def get_top_level(self):
    """
    Calculate top level time series by aggregating low level time-series
    """
    data = np.sum(self.df_train.values, axis=0)

    return data

  def get_exog_variables(self):
    self.df_calendar['snap_CA'] = self.df_calendar['snap_CA'].astype(int)
    self.df_calendar['snap_TX'] = self.df_calendar['snap_TX'].astype(int)
    self.df_calendar['snap_WI'] = self.df_calendar['snap_WI'].astype(int)
    calendar = self.df_calendar[['d', 'snap_CA','snap_TX','snap_WI', 'event_name_1', 'event_name_2']].copy()
    calendar['snap_count'] = calendar[['snap_CA','snap_TX','snap_WI']].apply(np.sum,axis=1)

    calendar['is_event_1'] = [isinstance(x , str)*1 for x in calendar['event_name_1']]
    calendar['is_event_2'] = [isinstance(x , str)*1 for x in calendar['event_name_2']]
    calendar['is_event'] = calendar[['is_event_1', 'is_event_2']].apply(np.sum, axis=1)
    calendar['is_event'] = np.where(calendar['is_event']>0,1,0)

    exog_fit = calendar[['snap_count', 'is_event']].iloc[:TEST_START-1,:]
    exog_fit.index = range(len(exog_fit))
    exog_pred = calendar[['snap_count', 'is_event']].iloc[TRAIN_START:TEST_START+self.horizon,:]
    exog_pred.index = range(len(exog_pred))

    exog_fit = TimeSeries.from_dataframe(exog_fit, value_cols=exog_fit.columns)
    exog_pred = TimeSeries.from_dataframe(exog_pred, value_cols=exog_fit.columns)

    return exog_fit, exog_pred

  def get_weights(self):
    w = np.sum(self.df_train.iloc[:, -28:].values, axis=1) / sum(self.top_level[-28:])
    w = w.reshape(len(w), 1)

    return w

  def predict_top_level(self):
    model = DLinearModel(
      # input_chunk_length=self.train_len - self.horizon,
      input_chunk_length=28,
      output_chunk_length=self.horizon,
      n_epochs=100,
      model_name='dlinear_model',
      random_state=42
    )

    y_train = TimeSeries.from_values(self.get_top_level())

    model.fit(
        y_train,
        # past_covariates=self.exog_fit,
        # future_covariates=self.exog_pred,
        verbose=False
    )
    preds = model.predict(
        self.horizon,
        # past_covariates=self.exog_fit,
        # future_covariates=self.exog_pred
    )

    return preds.values()

  def predict_bottom_level(self):
    w = self.get_weights()

    top_level_preds = self.predict_top_level()
    top_level_preds = top_level_preds.reshape(1,len(top_level_preds))
    preds = np.multiply(top_level_preds, w)
    item_ids = self.df_train.index
    res = pd.DataFrame({
        'id': item_ids
    })

    predictions_df = pd.DataFrame(preds, columns=[f'F{i+1}' for i in range(28)])
    preds = pd.concat([res['id'], predictions_df], axis=1)

    preds.set_index('id', inplace=True)

    return preds

In [ ]:
VER = 1.0
MODEL_NAME='28-day-lookback-100-epoch-no-params'
stores = global_df['store_id'].unique()
depts = global_df['dept_id'].unique()
stores, depts

(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
 Categories (10, object): ['CA_1', 'CA_2', 'CA_3', 'CA_4', ..., 'TX_3', 'WI_1', 'WI_2', 'WI_3'],
 ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']
 Categories (7, object): ['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1',
                          'HOUSEHOLD_2'])

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/data/m5-data/sample_submission.csv')
submission.set_index('id', inplace=True)
model_name = f"dlinear-{MODEL_NAME}-{VER}"
for store_id in stores:
  for dept_id in depts:
    df_train, df_calendar = get_data_by_store_id(store_id, dept_id)
    model = M5DLinear(horizon=28, df_train=df_train.iloc[:, 0:(TEST_START - TRAIN_START - 1)].copy(), df_calendar=df_calendar, epochs=100)
    preds = model.predict_bottom_level()

    submission.update(preds)

    del df_train
    del df_calendar
    del model
    gc.collect()

submission.reset_index(inplace=True)
submission.to_csv(f"/content/drive/MyDrive/data/m5-data/submissions/{model_name}.csv", index=False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
true = pd.read_csv('/content/drive/MyDrive/data/m5-data/sales_test_evaluation.csv')
df_train, df_calendar = get_data_by_store_id('CA_1', 'HOBBIES_1')

,item_id,dept_id,cat_id,store_id,state_id,d_1942,d_1943,d_1944,d_1945,d_1946,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2,0,1,0,0,...,2,1,2,0,0,1,0,1,3,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,2,0,1,0,...,1,0,0,1,0,0,2,1,1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,2,1,0,2,1,0,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,6,...,3,3,4,2,1,6,3,1,4,3
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2,0,1,1,2,...,0,1,2,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,HOBBIES_1_420,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,1,0,...,0,3,5,0,0,1,2,0,0,1
412,HOBBIES_1_421,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2,0,0,...,2,0,2,1,0,1,0,2,2,1
413,HOBBIES_1_422,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
414,HOBBIES_1_423,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,2,3,...,1,4,2,0,0,0,1,2,2,0


In [ ]:
model = M5DLinear(horizon=28, df_train=df_train.iloc[:, 0:1941].copy(), df_calendar=df_calendar, epochs=100)
preds = model.predict_bottom_level()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
cols_to_sum = [col for col in true.columns if col.startswith('d_')]
aggregated_sums = true[cols_to_sum].sum(axis=0)

In [ ]:
preds.loc['HOBBIES_1_001_CA_1_evaluation']

,HOBBIES_1_001_CA_1_evaluation
F1,1.262994
F2,0.935540
F3,0.976361
F4,0.918475
F5,1.273200
F6,1.329348
F7,1.421647
F8,1.109622
F9,1.345443
F10,1.201461


In [ ]:
true[(true['dept_id'] == 'HOBBIES_1') & (true['store_id'] == 'CA_1')]

,item_id,dept_id,cat_id,store_id,state_id,d_1942,d_1943,d_1944,d_1945,d_1946,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2,0,1,0,0,...,2,1,2,0,0,1,0,1,3,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,2,0,1,0,...,1,0,0,1,0,0,2,1,1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,2,1,0,2,1,0,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,6,...,3,3,4,2,1,6,3,1,4,3
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2,0,1,1,2,...,0,1,2,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,HOBBIES_1_420,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,1,0,...,0,3,5,0,0,1,2,0,0,1
412,HOBBIES_1_421,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2,0,0,...,2,0,2,1,0,1,0,2,2,1
413,HOBBIES_1_422,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
414,HOBBIES_1_423,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,2,3,...,1,4,2,0,0,0,1,2,2,0
